# Дополнительное ДЗ к уроку Basic

In [2]:
class Rectangle:
    def __init__(self, width, height):
        """Инициализация прямоугольника с шириной и высотой."""
        self.width = width
        self.height = height

    def area(self):
        """Возвращает площадь прямоугольника."""
        return self.width * self.height

    def scale(self, k):
        """Масштабирует стороны прямоугольника на коэффициент k."""
        self.width *= k
        self.height *= k


# Пример использования:
rect = Rectangle(4, 5)
print(rect.area())  # Вывод: 20
rect.scale(2)
print(rect.area())  # Вывод: 80

20
80


In [3]:
class Square(Rectangle):
    def __init__(self, side):
        """Инициализация квадрата с длиной стороны."""
        super().__init__(side, side)


# Пример использования:
square = Square(4)
print(square.area())  # Вывод: 16
square.scale(2)
print(square.area())  # Вывод: 64

16
64


In [4]:
def surface(shape):
    """Возвращает площадь объекта, у которого есть метод area()."""
    return shape.area()


# Проверка с Rectangle и Square
rect = Rectangle(4, 5)
square = Square(4)

print(surface(rect))   # Вывод: 20
print(surface(square)) # Вывод: 16

20
16


# Дополнительное ДЗ * к уроку Basic

In [5]:
from abc import ABC, abstractmethod

class BaseModel(ABC):
    @abstractmethod
    def fit(self, X, y):
        """Обучает модель на данных X и y."""
        raise NotImplementedError("Метод fit должен быть переопределён в подклассе")

    @abstractmethod
    def predict(self, X):
        """Делает предсказания на основе данных X."""
        raise NotImplementedError("Метод predict должен быть переопределён в подклассе")

In [6]:
from statistics import median

class MedianRegressor(BaseModel):
    def __init__(self):
        """Инициализация без хранения данных."""
        self.median_value = None

    def fit(self, X, y):
        """Сохраняет медиану значений y."""
        self.median_value = median(y)
        return self

    def predict(self, X):
        """Возвращает список из медианы той же длины, что и X."""
        if self.median_value is None:
            raise ValueError("Сначала вызовите метод fit()")
        return [self.median_value] * len(X)


# Пример использования:
X = [1, 2, 3]
y = [1, 2, 10]

model = MedianRegressor()
model.fit(X, y)
print(model.predict(X))  # Вывод: [2, 2, 2]

# Или вот так:
print(MedianRegressor().fit(X, [1,2,10]).predict(X)) # Вывод: [2, 2, 2]

[2, 2, 2]
[2, 2, 2]


In [7]:
import random
from collections import Counter

class PriorProbClassifier(BaseModel):
    def __init__(self):
        """Инициализация без хранения данных."""
        self.class_probs = None

    def fit(self, X, y):
        """
        Сохраняет априорные вероятности классов.
        :param X: данные (не используются, только для совместимости)
        :param y: метки классов
        """
        total = len(y)
        class_counts = Counter(y)
        self.class_probs = {label: count / total for label, count in class_counts.items()}
        return self

    def predict(self, X):
        """
        Предсказывает классы на основе априорных вероятностей.
        :param X: данные, для которых нужно сделать предсказания
        :return: список предсказанных классов
        """
        if self.class_probs is None:
            raise ValueError("Сначала вызовите метод fit()")
        labels, probs = zip(*self.class_probs.items())
        return random.choices(labels, probs, k=len(X))


# Пример использования:
X = [1, 2, 3, 4, 5]  # Данные (не используются)
y = ['A', 'A', 'B', 'A', 'B']  # Метки классов

model = PriorProbClassifier()
model.fit(X, y)
predictions = model.predict(X)

print("Априорные вероятности:", model.class_probs)  # Вывод: {'A': 0.6, 'B': 0.4}
print("Предсказания:", predictions)  # Пример: ['A', 'A', 'B', 'A', 'B']

Априорные вероятности: {'A': 0.6, 'B': 0.4}
Предсказания: ['A', 'A', 'A', 'B', 'B']


In [21]:
from sklearn.metrics import mean_absolute_error, accuracy_score

def evaluate(model, X, y, task='reg'):
    """
    Оценивает модель на основе задачи.
    :param model: модель, реализующая методы fit и predict
    :param X: входные данные
    :param y: истинные значения
    :param task: 'reg' для регрессии или 'clf' для классификации
    :return: метрика (MAE для регрессии, accuracy для классификации)
    """
    model.fit(X, y)
    predictions = model.predict(X)
    
    if task == 'reg':
        return mean_absolute_error(y, predictions)
    elif task == 'clf':
        return accuracy_score(y, predictions)
    else:
        raise ValueError("Неверное значение параметра task. Используйте 'reg' или 'clf'.")


# Пример использования:
X = [1, 2, 3, 4, 5]
y_reg = [1, 2, 10, 4, 5]  # Для регрессии
y_clf = ['A', 'A', 'B', 'A', 'B']  # Для классификации

# Оценка MedianRegressor
print(evaluate(MedianRegressor(), X, y_reg, task='reg'))  # Вывод: MAE

# Оценка PriorProbClassifier
print(evaluate(PriorProbClassifier(), X, y_clf, task='clf'))  # Вывод: accuracy

2.4
0.6


Подсказки по реализации
MedianRegressor — классический «baseline» для MAE-метрик; медиану трудно «побить» по средней абсолютной ошибке.
PriorProbClassifier — «честный» случайный базис, который учитывает несбалансированность классов и даёт интуитивный ориентир: если accuracy близка к частоте самого популярного класса, модель почти ничему не научилась.

# ДЗ advanced

In [22]:
class Point:
    def __init__(self, x, y):
        """Инициализация точки с координатами x и y."""
        self.x = x
        self.y = y

    def __add__(self, other):
        """Сложение двух точек."""
        if isinstance(other, Point):
            return Point(self.x + other.x, self.y + other.y)
        return NotImplemented

    def __sub__(self, other):
        """Вычитание двух точек."""
        if isinstance(other, Point):
            return Point(self.x - other.x, self.y - other.y)
        return NotImplemented

    def __mul__(self, scalar):
        """Умножение точки на скаляр."""
        if isinstance(scalar, (int, float)):
            return Point(self.x * scalar, self.y * scalar)
        return NotImplemented

    def __rmul__(self, scalar):
        """Правостороннее умножение точки на скаляр."""
        return self.__mul__(scalar)

    def __repr__(self):
        """Строковое представление точки."""
        return f"Point({self.x}, {self.y})"


# Пример использования:
p1 = Point(1, 2)
p2 = Point(3, 4)

# Сложение
print(p1 + p2)  # Вывод: Point(4, 6)

# Вычитание
print(p1 - p2)  # Вывод: Point(-2, -2)

# Умножение на скаляр
print(p1 * 3)   # Вывод: Point(3, 6)
print(3 * p1)   # Вывод: Point(3, 6) (правостороннее умножение)

Point(4, 6)
Point(-2, -2)
Point(3, 6)
Point(3, 6)


In [23]:
from functools import total_ordering

@total_ordering
class ModelScore:
    def __init__(self, name, score):
        """Инициализация модели с именем и оценкой."""
        self.name = name
        self.score = score

    def __eq__(self, other):
        """Проверка равенства моделей по их оценке."""
        if not isinstance(other, ModelScore):
            return NotImplemented
        return self.score == other.score

    def __lt__(self, other):
        """Сравнение моделей: меньше — если оценка меньше."""
        if not isinstance(other, ModelScore):
            return NotImplemented
        return self.score < other.score

    def __repr__(self):
        """Строковое представление модели."""
        return f"ModelScore(name='{self.name}', score={self.score})"


# Пример использования:
models = [
    ModelScore("Model A", 85),
    ModelScore("Model B", 92),
    ModelScore("Model C", 78),
    ModelScore("Model D", 92)
]

# Сортировка моделей от худшего к лучшему
sorted_models = sorted(models)
print("Отсортированные модели:", sorted_models)

# Модель с наивысшим score
best_model = max(models)
print("Лучшая модель:", best_model)

Отсортированные модели: [ModelScore(name='Model C', score=78), ModelScore(name='Model A', score=85), ModelScore(name='Model B', score=92), ModelScore(name='Model D', score=92)]
Лучшая модель: ModelScore(name='Model B', score=92)


In [24]:
class EvenSequence:
    def __init__(self, n):
        """Инициализация последовательности чётных чисел до n (не включая)."""
        self.n = n
        self.evens = list(range(0, n, 2))  # Генерация списка чётных чисел

    def __iter__(self):
        """Возвращает итератор для последовательности."""
        self._index = 0  # Сброс индекса для итерации
        return self

    def __next__(self):
        """Возвращает следующий элемент последовательности."""
        if self._index < len(self.evens):
            result = self.evens[self._index]
            self._index += 1
            return result
        else:
            raise StopIteration  # Завершение итерации

    def __len__(self):
        """Возвращает длину последовательности."""
        return len(self.evens)

    def __getitem__(self, index):
        """Возвращает элемент последовательности по индексу."""
        if 0 <= index < len(self.evens):
            return self.evens[index]
        else:
            raise IndexError("Индекс вне диапазона")


# Пример использования:
seq = EvenSequence(10)

# Итерация через for
for x in seq:
    print(x, end=" ")  # Вывод: 0 2 4 6 8
print()

# Длина последовательности
print(len(seq))  # Вывод: 5

# Индексирование
print(seq[2])  # Вывод: 4
try:
    print(seq[10])  # Вывод: IndexError
except IndexError as e:
    print(e)

0 2 4 6 8 
5
4
Индекс вне диапазона


In [26]:
class FileStats:
    def __init__(self, filepath):
        """Инициализация с указанием пути к файлу."""
        self.filepath = filepath
        self.file = None

    def __enter__(self):
        """Открывает файл и возвращает его."""
        self.file = open(self.filepath, 'r', encoding='utf-8')
        return self.file

    def __exit__(self, exc_type, exc_value, traceback):
        """Закрывает файл и выводит статистику."""
        if self.file:
            # Подсчёт строк и слов
            self.file.seek(0)  # Перемещаем указатель в начало файла
            lines = self.file.readlines()
            num_lines = len(lines)
            num_words = sum(len(line.split()) for line in lines)
            print(f"Lines: {num_lines}, Words: {num_words}")
            self.file.close()
        # Возвращаем False, чтобы исключения, если они были, не подавлялись
        return False


# Пример использования:
filepath = 'example.txt'  # Укажите путь к вашему текстовому файлу

with FileStats(filepath) as f:
    for line in f:
        print(line.strip())  # Чтение и вывод строк файла

scikit-learn
numpy
pandas
matplotlib
seaborn
scipy
tensorflow
torch torchvision torchtext
torchmetrics
transformers
Lines: 10, Words: 12


# 5★ SeededBatchLoader

In [27]:
import random

class SeededBatchLoader:
    def __init__(self, data: list, batch_size: int, shuffle: bool = True):
        """
        Инициализация загрузчика данных.
        :param data: список данных
        :param batch_size: размер батча
        :param shuffle: флаг перемешивания данных
        """
        self.data = data
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.current_index = 0

    def __iter__(self):
        """
        Возвращает итератор.
        Если shuffle=True, данные перемешиваются.
        """
        self.current_index = 0
        if self.shuffle:
            random.shuffle(self.data)
        return self

    def __next__(self):
        """
        Возвращает следующий батч данных.
        Если данных больше нет, вызывает StopIteration.
        """
        if self.current_index >= len(self.data):
            raise StopIteration
        batch = self.data[self.current_index:self.current_index + self.batch_size]
        self.current_index += self.batch_size
        return batch


# Пример использования:
data = [1, 2, 3, 4, 5, 6, 7, 8, 9]
batch_size = 3

# Создание загрузчика данных
loader = SeededBatchLoader(data, batch_size, shuffle=True)

# Итерация через батчи
for batch in loader:
    print(batch)  # Пример вывода: [4, 1, 7], [9, 2, 6], [3, 8, 5]

[6, 1, 3]
[9, 5, 7]
[4, 2, 8]


In [28]:
import random
import itertools

class SeededBatchLoader:
    def __init__(self, data: list, batch_size: int, shuffle: bool = True):
        """
        Инициализация загрузчика данных.
        :param data: список данных
        :param batch_size: размер батча
        :param shuffle: флаг перемешивания данных
        """
        self.data = data
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.current_index = 0

    def __iter__(self):
        """
        Возвращает итератор.
        Сбрасывает внутренний индекс и перемешивает данные, если shuffle=True.
        """
        self.current_index = 0
        if self.shuffle:
            random.shuffle(self.data)
        return self

    def __next__(self):
        """
        Возвращает следующий батч данных.
        Если данные закончились, начинает новую "эпоху".
        """
        if self.current_index >= len(self.data):
            # Начинаем новую "эпоху"
            self.current_index = 0
            if self.shuffle:
                random.shuffle(self.data)
        
        # Формируем батч
        batch = self.data[self.current_index:self.current_index + self.batch_size]
        self.current_index += self.batch_size
        return batch


# Пример использования:
data = [1, 2, 3, 4, 5, 6, 7, 8, 9]
batch_size = 3

# Создание загрузчика данных
loader = SeededBatchLoader(data, batch_size, shuffle=False)

# Проверка с itertools.islice
import itertools
print(list(itertools.islice(loader, 5)))  # Вывод: [[1, 2, 3], [4, 5, 6], [7, 8, 9], [1, 2, 3], [4, 5, 6]]

# Проверка с shuffle=True
loader = SeededBatchLoader(data, batch_size, shuffle=True)
print(list(itertools.islice(loader, 5)))  # Пример вывода: [[3, 1, 2], [6, 4, 5], [9, 7, 8], [3, 1, 2], [6, 4, 5]]

[[1, 2, 3], [4, 5, 6], [7, 8, 9], [1, 2, 3], [4, 5, 6]]
[[8, 2, 3], [7, 1, 6], [9, 5, 4], [4, 6, 3], [7, 2, 5]]


In [ ]:
import random

class SeededBatchLoader:
    def __init__(self, data: list, batch_size: int, shuffle: bool = True):
        """
        Инициализация загрузчика данных.
        :param data: список данных
        :param batch_size: размер батча
        :param shuffle: флаг перемешивания данных
        """
        self.data = data
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.current_index = 0

    def __iter__(self):
        """
        Возвращает итератор.
        Сбрасывает внутренний индекс и перемешивает данные, если shuffle=True.
        """
        self.current_index = 0
        if self.shuffle:
            random.shuffle(self.data)
        return self

    def __next__(self):
        """
        Возвращает следующий батч данных.
        Если данные закончились, начинает новую "эпоху".
        """
        if self.current_index >= len(self.data):
            # Начинаем новую "эпоху"
            self.current_index = 0
            if self.shuffle:
                random.shuffle(self.data)
        
        # Формируем батч
        batch = self.data[self.current_index:self.current_index + self.batch_size]
        self.current_index += self.batch_size
        return batch

    def __len__(self):
        """
        Возвращает количество батчей в одной эпохе.
        """
        return (len(self.data) + self.batch_size - 1) // self.batch_size  # Округление вверх


# Пример использования:
data = list(range(10))
batch_size = 4

# Создание загрузчика данных
loader = SeededBatchLoader(data, batch_size, shuffle=False)

# Количество батчей
print(len(loader))  # Вывод: 3 ([0-3], [4-7], [8-9])

# Итерация через батчи
for batch in loader:
    print(batch)  # Вывод: [0, 1, 2, 3], [4, 5, 6, 7], [8, 9]

In [ ]:
import random

class SeededBatchLoader:
    def __init__(self, data: list, batch_size: int, shuffle: bool = True):
        """
        Инициализация загрузчика данных.
        :param data: список данных
        :param batch_size: размер батча
        :param shuffle: флаг перемешивания данных
        """
        self.data = data
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.current_index = 0
        self._state = None  # Для сохранения состояния ГПСЧ

    def __iter__(self):
        """
        Возвращает итератор.
        Сбрасывает внутренний индекс и перемешивает данные, если shuffle=True.
        """
        self.current_index = 0
        if self.shuffle:
            random.shuffle(self.data)
        return self

    def __next__(self):
        """
        Возвращает следующий батч данных.
        Если данные закончились, начинает новую "эпоху".
        """
        if self.current_index >= len(self.data):
            # Начинаем новую "эпоху"
            self.current_index = 0
            if self.shuffle:
                random.shuffle(self.data)
        
        # Формируем батч
        batch = self.data[self.current_index:self.current_index + self.batch_size]
        self.current_index += self.batch_size
        return batch

    def __len__(self):
        """
        Возвращает количество батчей в одной эпохе.
        """
        return (len(self.data) + self.batch_size - 1) // self.batch_size  # Округление вверх

    def __enter__(self, seed=None):
        """
        Сохраняет текущее состояние ГПСЧ и устанавливает новый seed.
        """
        self._state = random.getstate()  # Сохраняем текущее состояние
        if seed is not None:
            random.seed(seed)  # Устанавливаем новый seed
        return self

    def __exit__(self, exc_type, exc_value, traceback):
        """
        Восстанавливает состояние ГПСЧ и не подавляет исключения.
        """
        random.setstate(self._state)  # Восстанавливаем сохранённое состояние
        return False  # Не подавляем исключения


# Пример использования:
data = [1, 2, 3, 4, 5, 6, 7, 8, 9]
batch_size = 3

# Сохранение состояния ГПСЧ
state_before = random.getstate()

# Создание загрузчика данных с seed
with SeededBatchLoader(data, batch_size, shuffle=True) as loader:
    random.seed(42)  # Устанавливаем seed
    print(list(loader))  # Пример вывода: [[3, 1, 2], [6, 4, 5], [9, 7, 8]]

# Проверка восстановления состояния ГПСЧ
state_after = random.getstate()
assert state_before == state_after  # Состояние должно быть восстановлено

In [35]:
import random

class SeededBatchLoader:
    def __init__(self, data: list, batch_size: int, shuffle: bool = True):
        """
        Инициализация загрузчика данных.
        :param data: список данных
        :param batch_size: размер батча
        :param shuffle: флаг перемешивания данных
        """
        self.data = data
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.current_index = 0

    def __iter__(self):
        """
        Возвращает итератор.
        Сбрасывает внутренний индекс и перемешивает данные, если shuffle=True.
        """
        self.current_index = 0
        if self.shuffle:
            random.shuffle(self.data)
        return self

    def __next__(self):
        """
        Возвращает следующий батч данных.
        Если данные закончились, начинает новую "эпоху".
        """
        if self.current_index >= len(self.data):
            # Начинаем новую "эпоху"
            self.current_index = 0
            if self.shuffle:
                random.shuffle(self.data)
        
        # Формируем батч
        batch = self.data[self.current_index:self.current_index + self.batch_size]
        self.current_index += self.batch_size
        return batch

    def __len__(self):
        """
        Возвращает количество батчей в одной эпохе.
        """
        return (len(self.data) + self.batch_size - 1) // self.batch_size  # Округление вверх

    def __enter__(self, seed=None):
        """
        Сохраняет текущее состояние ГПСЧ и устанавливает новый seed.
        """
        self._state = random.getstate()  # Сохраняем текущее состояние
        if seed is not None:
            random.seed(seed)  # Устанавливаем новый seed
        return self

    def __exit__(self, exc_type, exc_value, traceback):
        """
        Восстанавливает состояние ГПСЧ и не подавляет исключения.
        """
        random.setstate(self._state)  # Восстанавливаем сохранённое состояние
        return False  # Не подавляем исключения


# Пример использования:
data = [1, 2, 3, 4, 5, 6, 7, 8, 9]
batch_size = 3

# Создание загрузчика данных
loader = SeededBatchLoader(data, batch_size, shuffle=True)

# Бесконечная генерация батчей
import itertools
for i, batch in enumerate(itertools.islice(loader, 100)):
    print(f"Batch {i}: {batch}")
    if i == 5:  # Прерываем после 100 батчей
        break

Batch 0: [7, 3, 1]
Batch 1: [9, 6, 5]
Batch 2: [4, 8, 2]
Batch 3: [2, 5, 9]
Batch 4: [6, 1, 8]
Batch 5: [7, 4, 3]


# Проектируем иерархию классов для работы с файлами

In [32]:
from abc import ABC, abstractmethod
from datetime import datetime

# Базовый класс для медиа-файлов
class MediaFile(ABC):
    def __init__(self, name: str, size: int, created_at: datetime, owner: str):
        self.name = name
        self.size = size
        self.created_at = created_at
        self.owner = owner

    @abstractmethod
    def get_metadata(self):
        """Возвращает метаданные файла."""
        pass

    @abstractmethod
    def process(self):
        """Обрабатывает файл (например, конвертация, извлечение фич)."""
        pass

    def delete(self):
        """Удаляет файл."""
        print(f"Файл {self.name} удалён.")

# Подкласс для аудио-файлов
class AudioFile(MediaFile):
    def __init__(self, name: str, size: int, created_at: datetime, owner: str, duration: int, bitrate: int):
        super().__init__(name, size, created_at, owner)
        self.duration = duration  # Длительность в секундах
        self.bitrate = bitrate    # Битрейт в kbps

    def get_metadata(self):
        return {
            "name": self.name,
            "size": self.size,
            "created_at": self.created_at,
            "owner": self.owner,
            "duration": self.duration,
            "bitrate": self.bitrate,
        }

    def process(self):
        print(f"Конвертация аудио-файла {self.name}...")

# Подкласс для видео-файлов
class VideoFile(MediaFile):
    def __init__(self, name: str, size: int, created_at: datetime, owner: str, resolution: str, framerate: int):
        super().__init__(name, size, created_at, owner)
        self.resolution = resolution  # Разрешение (например, "1920x1080")
        self.framerate = framerate    # Частота кадров

    def get_metadata(self):
        return {
            "name": self.name,
            "size": self.size,
            "created_at": self.created_at,
            "owner": self.owner,
            "resolution": self.resolution,
            "framerate": self.framerate,
        }

    def process(self):
        print(f"Извлечение кадров из видео-файла {self.name}...")

# Подкласс для фото-файлов
class PhotoFile(MediaFile):
    def __init__(self, name: str, size: int, created_at: datetime, owner: str, resolution: str):
        super().__init__(name, size, created_at, owner)
        self.resolution = resolution  # Разрешение (например, "1920x1080")

    def get_metadata(self):
        return {
            "name": self.name,
            "size": self.size,
            "created_at": self.created_at,
            "owner": self.owner,
            "resolution": self.resolution,
        }

    def process(self):
        print(f"Применение фильтров к фото {self.name}...")

In [33]:
# Базовый класс для хранилищ
class Storage(ABC):
    @abstractmethod
    def save(self, file: MediaFile):
        """Сохраняет файл в хранилище."""
        pass

    @abstractmethod
    def load(self, file_name: str) -> MediaFile:
        """Загружает файл из хранилища."""
        pass

# Локальное хранилище
class LocalStorage(Storage):
    def save(self, file: MediaFile):
        print(f"Сохранение файла {file.name} на локальный диск.")

    def load(self, file_name: str) -> MediaFile:
        print(f"Загрузка файла {file_name} с локального диска.")
        # Возвращаем заглушку
        return AudioFile(file_name, 1024, datetime.now(), "user", 300, 128)

# Облачное хранилище
class CloudStorage(Storage):
    def save(self, file: MediaFile):
        print(f"Сохранение файла {file.name} в облако.")

    def load(self, file_name: str) -> MediaFile:
        print(f"Загрузка файла {file_name} из облака.")
        # Возвращаем заглушку
        return PhotoFile(file_name, 2048, datetime.now(), "user", "1920x1080")

In [34]:
# Создание файлов
audio = AudioFile("song.mp3", 5000, datetime.now(), "user1", 180, 320)
video = VideoFile("movie.mp4", 2000000, datetime.now(), "user2", "1920x1080", 30)
photo = PhotoFile("image.jpg", 3000, datetime.now(), "user3", "1920x1080")

# Работа с локальным хранилищем
local_storage = LocalStorage()
local_storage.save(audio)
loaded_audio = local_storage.load("song.mp3")
print(loaded_audio.get_metadata())

# Работа с облачным хранилищем
cloud_storage = CloudStorage()
cloud_storage.save(photo)
loaded_photo = cloud_storage.load("image.jpg")
print(loaded_photo.get_metadata())

# Обработка файлов
audio.process()
video.process()
photo.process()

Сохранение файла song.mp3 на локальный диск.
Загрузка файла song.mp3 с локального диска.
{'name': 'song.mp3', 'size': 1024, 'created_at': datetime.datetime(2025, 5, 10, 12, 11, 58, 185037), 'owner': 'user', 'duration': 300, 'bitrate': 128}
Сохранение файла image.jpg в облако.
Загрузка файла image.jpg из облака.
{'name': 'image.jpg', 'size': 2048, 'created_at': datetime.datetime(2025, 5, 10, 12, 11, 58, 185037), 'owner': 'user', 'resolution': '1920x1080'}
Конвертация аудио-файла song.mp3...
Извлечение кадров из видео-файла movie.mp4...
Применение фильтров к фото image.jpg...
